# Upload API folder to AWS S3 bucket

This is a temporary notebook for uploading all the files in the /api folder to a bucket on AWS S3

In [1]:
# Imports and general variables

import sys
from boto3 import Session
from pathlib import Path
from tqdm import tqdm

root_path = Path(globals()['_dh'][0]).resolve().parent
sys.path.append(str(root_path))

import paths


In [2]:
# Config

session = Session(profile_name="600627346413_PowerUser_with_billing")
s3 = session.client("s3", region_name="eu-central-1")  # Frankfurt region

bucket_name = 'energy-toolkit-vgr-data'

directory = Path(paths.api_path)

all_files = [file_path for file_path in directory.rglob('*') if file_path.is_file()]


In [3]:
# Delete existing files in bucket

paginator = s3.get_paginator("list_objects_v2")
for page in paginator.paginate(Bucket=bucket_name):
    if "Contents" in page:
        delete_keys = {"Objects": [{"Key": obj["Key"]} for obj in page["Contents"]]}
        s3.delete_objects(Bucket=bucket_name, Delete=delete_keys)

ClientError: An error occurred (InvalidToken) when calling the ListObjectsV2 operation: The provided token is malformed or otherwise invalid.

In [3]:
with tqdm(total=len(all_files), desc="Uploading files", unit="file") as pbar:
    for file_path in all_files:
        key = file_path.relative_to(directory).as_posix()

        with file_path.open("rb") as data:
            s3.upload_fileobj(data, bucket_name, key)

        pbar.update(1)
        pbar.set_postfix(file=key)

Uploading files: 100%|██████████| 950/950 [02:09<00:00,  7.36file/s, file=geography=1440-1489-1480-1401-1482-1441-1462-1481-1402-1415-1419-1407,target-year=2030,self-sufficiency=1,energy-scenario=30000000,h2=False,offwind=False,biogas-limit=0.025/performance/performance_metrics.csv.gz]        
